In [15]:
# Importing necessary libraries

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skimage.feature import hog, local_binary_pattern
from skimage import exposure
from sklearn.preprocessing import StandardScaler
from joblib import dump, load

import numpy as np
import pandas as pd
import cv2
import os

In [16]:
# Read data from CSV file
def read_data(csv_file):
    data = pd.read_csv(csv_file)
    return data

# Compute HOG for grayscale image
def compute_hog(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features = hog(gray_image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    return hog_features

# Calculate LBP for grayscale image
def compute_lbp(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(gray_image, P=24, R=3.0, method='uniform')
    return lbp_features.flatten()

# Function to train the model
def initialize_dataset(data):
    X = []
    y = []
    error_count = 0  # Count the number of errors when reading images
    for index, row in data.iterrows():
        image_path = row['image_path']
        
        # Check if the image file exists
        if not os.path.exists(image_path):
            print(f"File not found: {image_path}")
            error_count += 1
            continue
        
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error reading image: {image_path}")
            error_count += 1
            continue
        
        # Get the number of hands in the image
        num_hands = row['num_hands']
        
        # If there are no hands (num_hands = 0)
        if num_hands == 0:
            y.append([0, 0, 0, 0, 0, 0, 0, 0])  #  8 output variables for no hands
        
        # If there is 1 hand (num_hands = 1)
        elif num_hands == 1:
            x1_min = row['1_x_1']
            y1_min = row['1_y_1']
            x1_max = row['1_x_2']
            y1_max = row['1_y_2']
            y.append([x1_min, y1_min, x1_max, y1_max, 0, 0, 0, 0])  # 8 output variables for 1 hand
        # If there are 2 hands (num_hands = 2)
        elif num_hands == 2:
            x1_min = row['1_x_1']
            y1_min = row['1_y_1']
            x1_max = row['1_x_2']
            y1_max = row['1_y_2']
            x2_min = row['2_x_1']
            y2_min = row['2_y_1']
            x2_max = row['2_x_2']
            y2_max = row['2_y_2']
            y.append([x1_min, y1_min, x1_max, y1_max, x2_min, y2_min, x2_max, y2_max])  # 8 output variables for 2 hands
        
        # Compute HOG and LBP features
        hog_features = compute_hog(image)
        lbp_features = compute_lbp(image)

        # Add HOG and LBP features to the input data
        X.append(list(hog_features) + list(lbp_features))

        # Show progress
        print('Processing:', image_path)

    print(f"Number of images with errors: {error_count}")
    return X, np.array(y)


In [17]:
# CSV file path
csv_file = 'data/info.csv'

# Read data from CSV file
data = read_data(csv_file)

# Create the dataset
X, y = initialize_dataset(data)

Processing: data/images/thatlq_img_1.jpg
Processing: data/images/thatlq_img_2.jpg
Processing: data/images/thatlq_img_3.jpg
Processing: data/images/thatlq_img_4.jpg
Processing: data/images/thatlq_img_5.jpg
Processing: data/images/thatlq_img_6.jpg
Processing: data/images/thatlq_img_7.jpg
Processing: data/images/thatlq_img_8.jpg
Processing: data/images/thatlq_img_9.jpg
Processing: data/images/thatlq_img_10.jpg
Processing: data/images/thatlq_img_11.jpg
Processing: data/images/thatlq_img_12.jpg
Processing: data/images/thatlq_img_13.jpg
Processing: data/images/thatlq_img_14.jpg
Processing: data/images/thatlq_img_15.jpg
Processing: data/images/thatlq_img_16.jpg
Processing: data/images/thatlq_img_17.jpg
Processing: data/images/thatlq_img_18.jpg
Processing: data/images/thatlq_img_19.jpg
Processing: data/images/thatlq_img_20.jpg
Processing: data/images/thatlq_img_21.jpg
Processing: data/images/thatlq_img_22.jpg
Processing: data/images/thatlq_img_23.jpg
Processing: data/images/thatlq_img_24.jpg
P

In [18]:
print('Shape of X:', np.array(X).shape)
print('Shape of y:', y.shape)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Show the shape of the training and testing sets
print('Shape of X_train:', np.array(X_train).shape)
print('Shape of X_test:', np.array(X_test).shape)
print('Shape of y_train:', y_train.shape)
print('Shape of y_test:', y_test.shape)

Shape of X: (3256, 55836)
Shape of y: (3256, 8)
Shape of X_train: (2604, 55836)
Shape of X_test: (652, 55836)
Shape of y_train: (2604, 8)
Shape of y_test: (652, 8)


In [19]:
# Build the Random Forest model
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Save the model
model_file = 'models/hand_region_detection_model.pkl'
dump(rf_model, model_file)

['models/hand_region_detection_model.pkl']

In [20]:
# Predict the test set
y_pred = rf_model.predict(X_test)

# Accuracy of the model
for i in range(y_test.shape[1]):
    print(f"Variable {i + 1}:")
    print(classification_report(y_test[:, i], y_pred[:, i]))

Variable 1:
              precision    recall  f1-score   support

         0.0       0.48      1.00      0.65       141
         1.0       0.00      0.00      0.00         1
         2.0       0.00      0.00      0.00         1
         3.0       0.20      1.00      0.33         1
         4.0       0.00      0.00      0.00         2
         8.0       0.00      0.00      0.00         3
         9.0       0.00      0.00      0.00         0
        11.0       0.00      0.00      0.00         1
        12.0       0.00      0.00      0.00         2
        13.0       0.00      0.00      0.00         2
        14.0       0.00      0.00      0.00         1
        15.0       0.50      0.33      0.40         3
        16.0       0.00      0.00      0.00         2
        17.0       0.50      1.00      0.67         1
        18.0       0.00      0.00      0.00         0
        19.0       0.00      0.00      0.00         3
        20.0       0.00      0.00      0.00         2
        21.0   

f:\Anaconda3\envs\Py3915\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Anaconda3\envs\Py3915\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Anaconda3\envs\Py3915\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
f:\Anaconda3\envs\Py3915\lib\site-packages\sklea

In [21]:
# Load the model
rf_model = load(model_file)
"""
# Define a function to compute HOG features for a grayscale image
def compute_hog(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features = hog(gray_image, pixels_per_cell=(16, 16), cells_per_block=(1, 1))
    return hog_features

# Define a function to compute LBP features for a grayscale image
def compute_lbp(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(gray_image, P=16, R=2.0)
    return lbp_features.flatten()
"""
# Define a function to predict and visualize the result
def predict_and_visualize(image_path, model):
    # Đọc ảnh và tính toán đặc trưng
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 144))
    hog_features = compute_hog(image)
    lbp_features = compute_lbp(image)
    features = np.hstack((hog_features, lbp_features))
    
    # Predict
    predicted_positions = model.predict([features])[0]
    
    # Reshape kết quả dự đoán để đảm bảo nó là mảng 2D
    predicted_positions = np.reshape(predicted_positions, (-1, 4))
    print(predicted_positions)
    
    # Vẽ bounding box
    for pos in predicted_positions:
        x_min, y_min, x_max, y_max = pos.astype(int)
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    
    # Hiển thị kết quả
    cv2.imshow('Result: ', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Predict and return coordinates
def predict_and_return_coordinates(image, model):
    # Resize image and compute features
    resized_image = cv2.resize(image, (256, 144))
    hog_features = compute_hog(resized_image)
    lbp_features = compute_lbp(resized_image)
    features = np.hstack((hog_features, lbp_features))
    
    # Predict bounding box coordinates
    predicted_positions = model.predict([features])[0]
    
    # Reshape predicted positions to ensure it's a 2D array
    predicted_positions = np.reshape(predicted_positions, (-1, 4))
    
    # Scale predicted positions back to original image size
    height_ratio = image.shape[0] / resized_image.shape[0]
    width_ratio = image.shape[1] / resized_image.shape[1]
    
    scaled_positions = []
    for pos in predicted_positions:
        x_min, y_min, x_max, y_max = pos
        x_min = int(x_min * width_ratio)
        y_min = int(y_min * height_ratio)
        x_max = int(x_max * width_ratio)
        y_max = int(y_max * height_ratio)
        scaled_positions.append((x_min, y_min, x_max, y_max))
    
    return scaled_positions

# Function to process frames from camera
def process_camera(model):
    cap = cv2.VideoCapture(0)  # 0 for default camera, change if necessary

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 10)
    cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    
    if not cap.isOpened():
        print("Error opening camera")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error reading frame")
            break
        
        # Predict and get bounding box coordinates
        frame = cv2.resize(frame, (256,144))
        predicted_positions = predict_and_return_coordinates(frame, model)
        print(predicted_positions)
        
        # Draw bounding boxes
        for pos in predicted_positions:
            x_min, y_min, x_max, y_max = pos
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        
        # Display the resulting frame
        cv2.imshow('Hand Detection', frame)
        
        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()



In [22]:
# Test by random 5 images in test set use predict_and_return_coordinates function
for i in range(10):
    index = np.random.randint(0, len(X_test))
    image_path = data.iloc[index]['image_path']
    image = cv2.imread(image_path)
    predicted_positions = predict_and_return_coordinates(image, rf_model)
    print(predicted_positions)
    for pos in predicted_positions:
        x_min, y_min, x_max, y_max = pos
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    cv2.imshow('Result', image)
    cv2.waitKey(0)
cv2.destroyAllWindows()
    

[(13, 0, 99, 111), (131, 6, 221, 105)]
[(125, 3, 228, 120), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(40, 35, 117, 124), (0, 0, 0, 0)]
[(137, 33, 222, 134), (0, 0, 0, 0)]
[(13, 0, 99, 111), (131, 6, 221, 105)]
[(41, 15, 117, 109), (134, 14, 210, 105)]
[(38, 33, 117, 120), (0, 0, 0, 0)]
[(151, 16, 224, 101), (29, 21, 100, 96)]
[(0, 0, 0, 0), (0, 0, 0, 0)]


In [23]:
# Test by custom images
image_path = 'test2.jpg'
predict_and_visualize(image_path, rf_model)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [24]:
# Test by camera
process_camera(rf_model)

[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0), (0, 0, 0, 0)]
[(0, 0, 0, 0),

In [25]:
def process_camera(model):
    cap = cv2.VideoCapture(0)  # 0 for default camera, change if necessary

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
    
    if not cap.isOpened():
        print("Error opening camera")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error reading frame")
            break
        
        # Display the frame to verify
        cv2.imshow('Camera Frame', frame)
        
        # Predict and get bounding box coordinates
        predicted_positions = predict_and_return_coordinates(frame, model)
        print("Predicted positions:", predicted_positions)
        
        # Draw bounding boxes
        for pos in predicted_positions:
            x_min, y_min, x_max, y_max = pos
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        
        # Display the resulting frame
        cv2.imshow('Hand Detection', frame)
        
        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Test process_camera function
if __name__ == "__main__":
    rf_model = load(model_file)
    process_camera(rf_model)


Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
Predicted positions: [(0, 0, 0, 0), (0, 0, 0, 0)]
